In [1]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


date = 20220630

OD = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\05_MIB_OD\20220630\MIB_OD20220630_SJPP.txt', sep = "\t", header = 0, low_memory = False)
#Net Int Income Myr, Funds Charge Myr kat oracle

Product_Hierarchy = pd.read_csv (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\06_Product_Hierarchy\Product_Hierarchy.txt", sep = "\t", header = 0, low_memory = False)
Product_Hierarchy.columns = Product_Hierarchy.columns.str.replace(" ", "_")


location_cust = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\02_Teradata\01_Credit_Risk_Reporting\01_Cust\\"+str(date)+"\\S001_004_ACRM_CUST"+str(date)+".txt"
cust = pd.read_csv(location_cust, sep = "|", header = 0, error_bad_lines = False)
cust.columns = cust.columns.str.replace(" ", "_")

Total_Impaired  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)


In [2]:
OD1 = OD.iloc[np.where(OD['Type Of Loan'].isin([66,173,176,177,178,153]))]


OD1.columns = OD1.columns.str.replace(" ", "_")
OD1.columns = OD1.columns.str.upper()
OD1.columns = OD1.columns.str.strip()

OD2 = OD1.merge(cust[['CUS_NO','NAME']], on='CUS_NO', how='left')

Product_Hierarchy.columns = Product_Hierarchy.columns.str.upper()


OD3 = OD2.merge(Product_Hierarchy[['PRODUCT_HIERARCHY_CD','SOURCE_PROD_TYPE_DSC']],on='PRODUCT_HIERARCHY_CD',how='left')

In [10]:
Total_Impaired['IF_tag'] = 'Y'

sumA = Total_Impaired.fillna(0).groupby(['V_ORIGINAL_ACCOUNT_NUMBER','V_D_CUST_FIRST_NAME','IF_tag'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

sumB = Total_Impaired.fillna(0).groupby(['CUSTOMER_ID','V_D_CUST_FIRST_NAME','IF_tag'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()


In [17]:
#OD3.ACCOUNT_NO = OD3.ACCOUNT_NO.astype('Int64').astype('str')
#sumB.CUSTOMER_ID = sumB.CUSTOMER_ID.astype('Int64').astype('str')

OD_combine = OD3.merge(sumA.drop_duplicates().rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'ACCOUNT_NO'}),on='ACCOUNT_NO',how='left')

bygcif = OD3.merge(sumB.drop_duplicates().rename(columns={'CUSTOMER_ID':'CUS_NO'}),on='CUS_NO',how='left')

In [18]:
writer = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\OD_SJPP"+str(date)[:6]+"(python).xlsx", engine='xlsxwriter')


OD_combine.to_excel(writer, sheet_name='W Impaired by Account', index = False)

bygcif.to_excel(writer, sheet_name='W Impaired by gcif', index = False)


writer.save()

In [ ]:
#cross check

In [32]:
OD_SJPP = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\13_Adverse Portfolio\\"+str(date)[:4]+"\\"+str(date)+"\working\\breach\MIB_OD_SJPP"+str(date)+"(python).xlsx")
#Net Int Income Myr, Funds Charge Myr kat oracle

In [33]:
OD_SJPP.head(1)

,ACCOUNT_NO,CUS_NO,AMRT_TERM,AMRT_TERM_MULT,AS_OF_DATE,BRANCH_CODE,COMMON_COA_ID,RECORD_COUNT,ACCT_STATUS_CD,ACCOUNT_OPEN_DATE,DATE_AA_APPROVED,AUTHORISED_LIMIT,DRAWING_LIMIT,SOURCE_PROD_TYPE_CD,SPECIAL_MENTION_ACCT_FLG,INTEREST_INCOME,INTEREST_INCOME_MYR,NON_INTEREST_INCOME,INTEREST_IN_SUSPENSE,SPECIFIC_PROVISION,ECONOMIC_SECTOR_CD,FULL_WRITEOFF_FLG,RESCHEDULE_FLG,RESTRUCTURE_FLG,NPL_FLG,NEW_NPL_FLG,TOTAL_ACTIVITY_COST,MAINTENANCE_COST,ENTITY_CD,M_BNM_BALANCE,M_SUB_MARKET_SEGMENT_CD,PRODUCT_HIERARCHY_CD,M_FINANCING_CONCEPT,M_MTHS_IN_ARREARS,AVG_BOOK_BAL,FUNDS_CHARGE,NET_INT_INCOME,CUR_GROSS_RATE,AA_NUMBER,GL_ACCOUNT_ID,FIRST_RELEASE_DATE,FINAL_RELEASE_DATE,RESCHEDULE_DATE,RESTRUCTURE_DATE,NPL_DATE,SECURITY_TYPE_CD,TYPE_OF_LOAN,NAME,SOURCE_PROD_TYPE_DSC
0,501132133336,9742084,60,M,30-JUN-2022,113,12000,1,1,05-APR-2001,03-DEC-2019,1000000.0,1000000.0,PMA,N,1472.58,1472.58,80.0,0.0,0.0,3693,N,N,N,N,N,10.35,31.5,MIB,295749.2,2,PMA_66_12000,NaN,0,259706.05,518.7,953.88,6.9,2019JZ1347,120101212119,27-MAR-2020,27-MAR-2020,01-JAN-1900,01-JAN-1900,01-JAN-1900,12,66,GOOD WIN CONCRETE SDN. BHD.,WCGS


In [7]:
Total_Impaired = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(date)[:4]+"\\"+str(date)[:6]+"\working\(MIB) Total_impaired_"+str(date)[:6]+".xlsx")


In [25]:
Total_Impaired.head(1)

,N_RUN_SKEY,V_ENTITY_CODE,N_ACCT_SKEY,V_ACCOUNT_NUMBER,V_ORIGINAL_ACCOUNT_NUMBER,CUSTOMER_ID,V_D_CUST_FIRST_NAME,F_EXPOSURE_DEFAULT_STATUS_FLAG,F_IMPAIRMENT_STATUS,F_IA_IND,PREV_STAGE_IFRS,CURR_STAGE_IFRS,V_CORPORATE_REMEDIAL_CD,MKT_SUB_SEGMENT,V_PROD_GROUP_DESC,PROD_LV1,PROD_LV2,PROD_LV3,PROD_LV4,PROD_LV5,PROD_LV6,V_ACCOUNT_CLASSIFICATION_CODE,OUTSTANDING,EIR,AMORT_COST,CARRYING_AMT,DRAWN_AMT,UNDRAWN_AMT,EAD_BEFORE_MOD,MOD_GAIN_LOSS,REMAINING_MOD_GAIN_LOSS,EAD_AFTER_MOD,ECL_OPENING_ORI_CCY,ECL_OPENING_BAL_REVAL,ECL_OPENING_1ST_JAN,ECL_CHARGED,ECL_WRITE_BACK,ECL_WRITE_OFF,ECL_FX_DIFF,ECL_CLOSING_BAL,NET_ECL_PL,INDIVIDUAL_ALLOWANCE,DCF_AMT,UWI_OPENING_ORI_CCY,UWI_OPENING_BAL_REVAL,UWI_OPENING_1ST_JAN,UWI_CHARGED,UWI_WRITE_BACK,UWI_WRITE_OFF,UWI_FX_DIFF,UWI_CLOSING_BAL,UWI_FTM,WRITE_OFF_AMOUNT_RCY,WRITE_OFF_YTD,WRITE_OFF_AMOUNT,EXPOSURE_LIMIT,LEDGER_BAL,TOTAL_12M_ECL_BFORE_FL,TOTAL_12M_ECL_AFTER_FL,TOTAL_LIFETIME_BFORE_FL,TOTAL_LIFETIME_AFTER_FL,TOTAL_ECL_BFORE_FL,TOTAL_ECL_AFTER_FL,DEFAULT_RULE,COUNT_RECORDS,V_GL_CODE,CURR_RATING,ORIG_RATING,CURR_RATING_BAND_CATEGORY,CURR_RATING_BAND,ECL_METHOD,RS_INCLUSION,V_CCY_CODE,V_RELEASE_TYPE_CD,F_RPSIA_FLAG,SMF_FLAG,V_SEGMENT_ID,N_MONTH_IN_ARREARS,N_DELINQUENT_DAYS,V_LEVEL5_LOB_CODE,MIS_BY_PURPOSE,V_D_CUST_INDUSTRY,MIS_BY_NOB,V_PROD_CODE,V_PROD_DESC,V_PROD_FAMILY_DESC,MIS_BY_CONCEPT,V_CLASS_CODE,V_D_CUST_TYPE,MIS_BY_CUSTOMER_TYPE,V_PURPOSE_DESC,V_BUSINESS_UNIT_CODE,V_BUSINESS_UNIT_DESC,V_REGION_CODE,V_ACCT_BRANCH_CODE,V_REGION_DESC,V_D_ACCOUNT_STATUS,N_RCY_CONVERSION_FACTOR,N_TWELVE_MONTHS_PD,N_LIFETIME_PD,N_LGD_PERCENT,V_BASEL_LGD_POOL_ID,N_EFFECTIVE_INTEREST_RATE,N_CURR_INTEREST_RATE,V_INTEREST_METHOD,N_INTEREST_FREQ,V_INTEREST_FREQ_UNIT,SUM(A.N_INT_INCOME),SUM(A.N_INT_INCOME_YTD_RCY),F_AKPK_FLAG,D_AKPK_DATE,D_FINAL_DISBURSEMENT_DATE,D_ISSUE_DATE,D_LAST_PAYMENT_DATE,D_LAST_DELINQUENT_DATE,D_LATEST_DELINQUENT_DATE,F_MODIFICATION_FLAG,D_MODIFICATION_DATE,F_COMMITMENT_FLAG,F_SMA_FLAG,WL_FLAG,N_CCF_PERCENT,N_DRAWN_CCF_PERCENT,SUM(A.N_SANCTION_AMOUNT),N_DISCOUNTING_FACTOR,F_EXP_REVOLVING_FLAG,F_EMP_ACCT_FLAG,F_NEW_ACCT_FLG,F_DISBURSED_IND,F_RESCHEDULE_IND,D_RESCHEDULE_DATE,F_RESTRUCTURED_IND,D_RESTRUCTURED_DATE,F_REMOVE_RESCHEDULE_IND,F_REMOVE_RESTRUCTURE_IND,F_REPACKAGING_IND,D_REPACKAGE_DATE,F_REPRICE_FLAG,D_REPRICE_DATE,CORPORATE_SCORECARD,V_SECTOR_CATEGORY,V_PD_TERM_STRUCTURE_ID,V_LGD_TERM_STRUCTURE_ID,N_ACCOUNT_TENOR,F_BANKRUPTCY_FLAG,F_REMOVE_BANKRUPTCY_FLAG,MATURITY_DATE,COLLATERAL_BAND,NPL_STATUS,F_PREF_TREATMENT_FLAG,GIL,V_STAGE_REASON_DESC,GIL DEC-21,STATUS,checking,ECL_WRITE_OFF_ (FINAL),STATUS\nWoff,No,last month,TRUE FALSE,GIL-Apr21,Var- FTM,Unnamed: 162
0,19156,MIB,3379790,139415546,764016811075,19492758,TRURICH RESOURCES SDN BHD,Y,Y,NaN,NaN,3,N,CORPORATE BANKING,LOANS_OTH,On Balance Sheet,Assets,Loans And Advances,Term Loans,Others,TERM FINANCING-I,AMRTCOST,4.03e+08,0.0,4.03e+08,7.52e+07,3.52e+08,0.0,4.03e+08,280714.37,280714.37,4.04e+08,6.72e+07,2.96e+08,2.80e+08,0.0,3.18e+06,0.0,1.58e+07,2.93e+08,-3.18e+06,3.28e+08,7.57e+07,6.55e+06,2.89e+07,2.73e+07,6.19e+06,0.0,0.0,1.54e+06,3.51e+07,1.43e+06,0.0,0.0,NaN,3.53e+08,4.03e+08,NaN,0,1.72e+08,1.72e+08,2.93e+08,3.28e+08,N,1,120301210114,R23,R13,MAYBANK RATING,NPL,Specific Provision Methodology,NaN,USD,NaN,N,N,MY_Non-Retail,30.0,901.0,1120.0,WORKING CAPITAL,1113,"AGRICULTURE,HUNTING AND FORESTRY",H1_OTHLNS,TERM FINANCING-I,H1,MURABAHAH,121,201,Domestic Business Enterprise -Others,NaN,26471,BR KL MAIN OFFICE,R08,6401,FEDERAL TERRITORY,Active,0.23,1.0,1,0.42,NaN,5.24,5.24,NaN,NaN,NaN,330324.1,1.74e+06,N,01-JAN-00,22-AUG-17,14-AUG-17,NaN,NaN,NaN,Y,31-MAR-19,N,NaN,NaN,0.0,NaN,NaN,NaN,N,N,NaN,NaN,N,NaN,N,NaN,NaN,NaN,Y,08-FEB-19,N,01-JAN-00,OH,07,NaN,MY_Non-Retail_CB_U,59,N,NaN,14-AUG-27,UNSECURED,NPL THRESHOLD NOT EXCEEDED,NaN,3.69e+08,Impairment Status Y,3.48e+08,Amount recovered,NaN,0.0,0,NaN,Amount recovered,NaN,NaN,NaN,NaN


In [24]:
OD_SJPP.ACCOUNT_NO = OD_SJPP.ACCOUNT_NO.astype(str)
OD_SJPP.ACCOUNT_NO = OD_SJPP.ACCOUNT_NO.str.strip()
OD_SJPP.ACCOUNT_NO = OD_SJPP.ACCOUNT_NO.str.upper()

Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.strip()
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.upper()


#value.toString().split(".")[0]

In [35]:
OD_SJPP.ACCOUNT_NO = OD_SJPP.ACCOUNT_NO.astype('Int64').astype('str')

In [34]:
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.dtypes, OD_SJPP.ACCOUNT_NO.dtypes

(dtype('O'), dtype('int64'))

In [37]:
a = OD_SJPP.merge(Total_Impaired[['V_ORIGINAL_ACCOUNT_NUMBER']].drop_duplicates().rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'ACCOUNT_NO'}),\
                 on='ACCOUNT_NO',how='left',indicator='IF_tag')

In [42]:
writer2 = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\13_Adverse Portfolio\\"+str(date)[:4]+"\\"+str(date)+"\working\\breach\MIB_OD_SJPP"+str(date)+"(python).xlsx",engine='xlsxwriter')

a.to_excel(writer2, sheet_name='SJPP', index = False)

writer2.save()

In [38]:
a.IF_tag.value_counts()

left_only     1302
both            25
right_only       0
Name: IF_tag, dtype: int64

In [39]:
b=a.iloc[np.where(a.IF_tag=='left_only')]
b.NPL_FLG.value_counts()

N    1302
Name: NPL_FLG, dtype: int64

In [40]:
c=a.iloc[np.where(a.IF_tag=='both')]
c.NPL_FLG.value_counts()

Y    22
N     3
Name: NPL_FLG, dtype: int64

In [41]:
c.sort_values(by='NPL_FLG',ascending=True)

,ACCOUNT_NO,CUS_NO,AMRT_TERM,AMRT_TERM_MULT,AS_OF_DATE,BRANCH_CODE,COMMON_COA_ID,RECORD_COUNT,ACCT_STATUS_CD,ACCOUNT_OPEN_DATE,DATE_AA_APPROVED,AUTHORISED_LIMIT,DRAWING_LIMIT,SOURCE_PROD_TYPE_CD,SPECIAL_MENTION_ACCT_FLG,INTEREST_INCOME,INTEREST_INCOME_MYR,NON_INTEREST_INCOME,INTEREST_IN_SUSPENSE,SPECIFIC_PROVISION,ECONOMIC_SECTOR_CD,FULL_WRITEOFF_FLG,RESCHEDULE_FLG,RESTRUCTURE_FLG,NPL_FLG,NEW_NPL_FLG,TOTAL_ACTIVITY_COST,MAINTENANCE_COST,ENTITY_CD,M_BNM_BALANCE,M_SUB_MARKET_SEGMENT_CD,PRODUCT_HIERARCHY_CD,M_FINANCING_CONCEPT,M_MTHS_IN_ARREARS,AVG_BOOK_BAL,FUNDS_CHARGE,NET_INT_INCOME,CUR_GROSS_RATE,AA_NUMBER,GL_ACCOUNT_ID,FIRST_RELEASE_DATE,FINAL_RELEASE_DATE,RESCHEDULE_DATE,RESTRUCTURE_DATE,NPL_DATE,SECURITY_TYPE_CD,TYPE_OF_LOAN,NAME,SOURCE_PROD_TYPE_DSC,IF_tag
1072,563064145599,16794703,60,M,30-JUN-2022,6306,12000,1,1,23-MAY-2018,26-JAN-2021,157500.00,157500.00,ICA,N,1028.85,1028.85,0.0,0.00,0.00,9999,N,N,N,N,N,3.85,31.50,MIB,158460.41,2,ICA_66_12000,3.0,0,158162.52,315.89,712.96,7.91,2022KW1002,120101212119,28-MAY-2018,25-MAR-2021,01-JAN-1900,01-JAN-1900,01-JAN-1900,59,66,DACING SEMPURNA SDN. BHD.,WCGS,both
1033,563028622392,24662612,60,M,30-JUN-2022,6302,12000,1,7,23-JAN-2018,08-NOV-2018,475000.00,475000.00,ICA,Y,3245.39,3245.39,0.0,0.00,0.00,9999,N,N,N,N,N,0.00,31.50,MIB,555561.14,2,ICA_66_12000,3.0,1,552423.93,1103.33,2142.06,7.15,2018KY1098,120101212119,28-MAR-2018,28-MAR-2018,01-JAN-1900,01-JAN-1900,01-JAN-1900,64,66,ALB TECHNOLOGIES SDN. BHD.,WCGS,both
728,562142422969,15675935,60,M,30-JUN-2022,6214,12000,1,2,06-AUG-2019,17-JUN-2019,306897.44,306897.44,ICA,N,2167.14,2167.14,0.0,0.00,0.00,7191,N,N,N,N,N,3.74,10.50,MIB,303342.15,1,ICA_66_12000,3.0,0,305047.25,609.26,1557.88,8.64,2019VH1417,120101212119,26-SEP-2019,26-SEP-2019,01-JAN-1900,01-JAN-1900,01-JAN-1900,12,66,MANDIRI SEJAHTERA SDN BHD,WCGS,both
27,551043521419,18596699,60,M,30-JUN-2022,5104,12000,1,1,06-DEC-2017,07-JAN-2020,125000.00,0.00,ICA,N,1416.70,1416.70,200.0,34907.71,614056.27,9999,N,N,N,Y,N,16.70,42.00,MIB,648964.98,2,ICA_66_12000,3.0,21,648202.17,2024.52,-607.82,2.66,2019JV1139,120211212119,16-JAN-2018,16-JAN-2018,01-JAN-1900,01-JAN-1900,31-DEC-2020,12,66,ROXY HERITAGE FURNITURE MANUFACTURER SDN,WCGS,both
1065,563055232742,23593706,60,M,30-JUN-2022,6305,12000,1,1,08-NOV-2018,23-AUG-2018,428000.00,0.00,ICA,N,381.97,381.97,0.0,6166.61,460457.52,8329,N,N,N,Y,N,0.00,42.00,MIB,466625.13,2,ICA_153_12000,3.0,21,466255.89,1456.25,-1074.28,1.00,2018KY1070,120201212119,26-NOV-2018,26-NOV-2018,01-JAN-1900,01-JAN-1900,31-DEC-2020,59,153,MUKSYN ENGINEERING & SERVICES SDN. BHD.,SERVICES SECTOR GUARANTEE SCHEME,both
1013,562991014699,25960138,60,M,30-JUN-2022,6299,12000,1,1,22-JAN-2019,04-OCT-2019,328427.45,0.00,ICA,N,283.29,283.29,0.0,6733.53,338944.24,3819,N,N,N,Y,N,0.00,15.75,MIB,345678.77,1,ICA_66_12000,3.0,4,345404.92,1078.80,-795.51,1.00,2019VC1400,120201212119,24-JAN-2019,24-JAN-2019,01-JAN-1900,01-JAN-1900,26-JAN-2022,59,66,SEPEN ENGINEERING SDN BHD,WCGS,both
1001,562982007997,25832020,60,M,30-JUN-2022,6298,12000,1,1,09-OCT-2018,05-MAR-2019,245699.73,0.00,ICA,N,2152.70,2152.70,0.0,1501.10,249171.29,6250,N,N,N,Y,N,0.00,15.75,MIB,250673.39,1,ICA_66_12000,3.0,3,248592.45,776.43,1376.27,10.54,2019VH1105,120211212119,16-APR-2019,16-APR-2019,01-JAN-1900,01-JAN-1900,30-SEP-2021,12,66,ZSN HOLDINGS SDN. BHD.,WCGS,both
950,562825057627,20867880,60,M,30-JUN-2022,6282,12000,1,1,29-NOV-2018,15-OCT-2018,125000.00,0.00,ICA,N,946.76,946.76,0.0,6903.60,191058.72,8329,N,N,N,Y,N,0.00,15.75,MIB,197963.32,1,ICA_153_12000,3.0,18,197048.12,615.44,331.32,5.85,2018VH1611,120211212119,30-NOV-2018,30-NOV-2018,01-JAN-1900,01-JAN-1900,02-APR-2021,12,153,SAFIR GO TRAVEL SDN BHD,SERVICES SECTOR GUARANTEE SCHEME,both
889,562674241776,27282457,60,M,30-JUN-2022,6267,12000,1,1,15-JUN-2021,08-MAY-2021,300000.00,300000.00,ICA,N,2392.76,2392.76,0.0,7164.80,311452.12,6250,N,N,N,Y,N,0.00,15.75,MIB,318617.92,1,ICA_66_12000,3.0,6,316304.92,987.91,1404.85,9.20,2021VC1230,120201212119,25-JUN-2021,25-JUN-